**Using `secret = xd`**

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
secret = os.getenv('SECRET_KEY')
print(f"secret:{secret}")

secret:this can be anything. update it later.


In [2]:
import requests
import time 
import hashlib

In [45]:

def good_hash_header():
    tt = str(time.time_ns())
    hash_ = hashlib.new('sha512')
    hash_.update(f"{tt}-{secret}".encode())
    header = {'time-token': tt, 'auth-key': hash_.hexdigest()}
    return header
    

### all clear test case

In [4]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": 0,
  "lng": 0
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

{'lat': 0, 'lng': 0}
{"job_ids":[10,11,1,11]}


### wrong/no header test case

In [23]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)

#header = good_hash_header() 
#time.sleep(6)
x = requests.post(url, json = body)

print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 1"}


### late hash test case

In [24]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)

header = good_hash_header() 
time.sleep(6)
x = requests.post(url, json = body, headers = header)

print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 2"}


### duplicate hash test case

In [25]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)

header = good_hash_header()
x = requests.post(url, json = body, headers = header)
x = requests.post(url, json = body, headers = header)

print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 3"}


### bad hash test case

In [26]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)

header = good_hash_header()
header['time-token'] = str(time.time_ns()) # by the time this runs, it mismatches the hash
x = requests.post(url, json = body, headers = header)

print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 4"}


In [27]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)
secret, secret_temp = 'NOT LONGER XD', secret
header = good_hash_header()
header['time-token'] = str(time.time_ns()) # by this runs, it will mismatches the hash
x = requests.post(url, json = body, headers = header)

secret, secret_temp = secret_temp, secret
print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 4"}


### edit functionality testing 

In [5]:
url = 'http://127.0.0.1:8000/add_job'
body = {'id_': 11,
  "lat": -32,
  "lng": 4
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

url = 'http://127.0.0.1:8000/edit_job'
body = { 'id_': 11,
"lat_old": -32,
"lng_old": 4,
"lat_new": 0,
"lng_new": 0
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

{'id_': 11, 'lat': -32, 'lng': 4}
{"message":3}
{'id_': 11, 'lat_old': -32, 'lng_old': 4, 'lat_new': 0, 'lng_new': 0}
{"message":"11 no longer in 87ad4536cffffff. 11 added to 87754e64dffffff. Length of 87754e64dffffff now 14"}


In [32]:
url = 'http://127.0.0.1:8000/add_job'
body = {'id_': 11,
  "lat": -32,
  "lng": 4
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

url = 'http://127.0.0.1:8000/del_job'
body = { 'id_': 11,
"lat": -32,
"lng": 4,
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

{'id_': 11, 'lat': -32, 'lng': 4}
{"message":3}
{'id_': 11, 'lat': -32, 'lng': 4}
{"message":"11 no longer in 87ad4536cffffff."}


## analytics testing

In [145]:
url = 'http://127.0.0.1:8000/submit_analytics'
body = {'feeds':  ## rename this?
        {
            'click/impression':{'impressions':[0,1,2], 'applies':[0], 'views':[0], 'saves':[0]},
            'applies/impression':{'impressions':list(range(10)), 'applies':[], 'views':[], 'saves':[3]},
        }
       }
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())
print(x.text)

{'feeds': {'click/impression': {'impressions': [0, 1, 2], 'applies': [0], 'views': [0], 'saves': [0]}, 'applies/impression': {'impressions': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'applies': [], 'views': [], 'saves': [3]}}}
{"mesage":"Received"}


## TESTING request_rec_feed endpoint

In [ ]:
# clear redis 
# add 50 jobs on 0,0 
# add 50 jobs on at 4,4
# use DBs

In [89]:
import redis
import h3
import numpy as np

In [96]:
#clear db
r = redis.Redis('localhost') 
for key in r.scan_iter("prefix:*"):
    r.delete(key)

In [103]:
url = 'http://127.0.0.1:8000/add_job'

for i in range(100):
    if i < 50:
        body = {'id_': i,
          "lat": 0,
          "lng": 0
        }
    else:
        body = {'id_': i,
          "lat": 4,
          "lng": 4
        }
    #print(body)
    x = requests.post(url, json = body, headers = good_hash_header())
    

In [131]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": 0,
  "lng": 0
}
x = requests.post(url, json = body, headers = good_hash_header())
x.text

'{"job_ids":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49]}'

In [146]:
url = 'http://127.0.0.1:8000/request_rec_feed'
body = {
  "lat": 0,
  "lng": 0
}
x = requests.post(url, json = body)
x.text

'{"rec_feed":[21,23,24,25,26,27,28,22,30,33,34,35,36,38,39,31,41,48,17,16,15,14,13,10,18,9,1,2,4,0,3,8,5,6,7,11,12,19,20,29,32,37,40,42,43,44,45,46,47,49]}'